In [9]:
%matplotlib inline
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

### Nhanes dataset

In [5]:
#Read data
da = pd.read_csv('nhanes_2015_2016.csv')

#Isolate needed data and drop missing data
vars = ["BPXSY1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", "DMDEDUC2", "BMXBMI",
        "SMQ020", "SDMVSTRA", "SDMVPSU"]
da = da[vars].dropna()

da.head()

,BPXSY1,RIDAGEYR,RIAGENDR,RIDRETH1,DMDEDUC2,BMXBMI,SMQ020,SDMVSTRA,SDMVPSU
0,128.0,62,1,3,5.0,27.8,1,125,1
1,146.0,53,1,3,3.0,30.8,1,125,1
2,138.0,78,1,3,3.0,28.8,1,131,1
3,132.0,56,2,3,5.0,42.4,2,131,1
4,100.0,42,2,4,4.0,20.3,2,126,2


In [6]:
#Obtaining MVU identifiers to be used as clusters
da['group'] = 10*da.SDMVSTRA + da.SDMVPSU

Similarity among observations within a cluster can be measured using a statistic called the intraclass correlation or ICC. We can assess ICC using two regression techniques, _marginal regression_, and _multilevel regression_.

In [14]:
#ICC for systollic blood pressure using GEE (marginal)

model = sm.GEE.from_formula(
    formula = 'BPXSY1 ~ 1',
    groups = 'group',
    cov_struct = sm.cov_struct.Exchangeable(),
    data = da
)

result = model.fit()

print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.030


In [16]:
#Recode smoking as a binary outcome
da['smq'] = da['SMQ020'].replace({2:0, 7:np.nan, 9:np.nan})

#ICC for the other variables
for v in ["BPXSY1", "RIDAGEYR", "BMXBMI", "smq", "SDMVSTRA"]:
    model = sm.GEE.from_formula(
        formula = v + " ~ 1",
        groups = 'group',
        cov_struct = sm.cov_struct.Exchangeable(),
        data = da
    )
    result = model.fit()
    print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.030
The correlation between two observations in the same cluster is 0.035
The correlation between two observations in the same cluster is 0.039
The correlation between two observations in the same cluster is 0.026
The correlation between two observations in the same cluster is 0.959


The ICC for SDMVSTRA is very high because it's part of the cluster definition

In [17]:
#Compare with ICC for 10 random sets with no dependence
for k in range(10):
    da['noise'] = np.random.normal(size = da.shape[0])
    model = sm.GEE.from_formula(
        formula = 'noise ~ 1',
        groups = 'group',
        cov_struct = sm.cov_struct.Exchangeable(),
        data = da
    )
    result = model.fit()
    print(result.cov_struct.summary())


The correlation between two observations in the same cluster is 0.001
The correlation between two observations in the same cluster is 0.001
The correlation between two observations in the same cluster is 0.002
The correlation between two observations in the same cluster is -0.001
The correlation between two observations in the same cluster is -0.000
The correlation between two observations in the same cluster is 0.002
The correlation between two observations in the same cluster is 0.001
The correlation between two observations in the same cluster is -0.001
The correlation between two observations in the same cluster is 0.001
The correlation between two observations in the same cluster is -0.001


The ICC's studied above were marginal, in the sense that we were looking at whether, say, the SBP values were more similar within versus between clusters. To the extent that such "cluster effects" are found, it may be largely explained by demographic differences among the clusters. For example, we know from our previous analyses with the NHANES data that older people have higher SBP than younger people. Also, some clusters may contain a slightly older or younger set of people than others. Thus, by controlling for age, we might anticipate that the ICC will become smaller.

In [19]:
#Controling for age
model = sm.GEE.from_formula(
    formula = 'BPXSY1 ~ RIDAGEYR',
    groups = 'group',
    cov_struct = sm.cov_struct.Exchangeable(),
    data = da
)

result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.019


We can now assess whether it drops even further when we add additional covariates that we know to be predictive of blood pressure.

In [22]:
#Relabelling gender variable
da['RIAGENDRx'] = da.RIAGENDR.replace({1:'Male', 2:'Female'})

#RIDRETH1 is a categorical variable coded numerically, thus we must use the C() syntax 
#in the formula to force this variable to be treated as being categorical.
model = sm.GEE.from_formula(
    formula = "BPXSY1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(RIDRETH1)",
    groups = 'group',
    cov_struct = sm.cov_struct.Exchangeable(),
    data = da
)

result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.013


### Marginal linear model with dependent data

In [27]:
#OLS vs GEE standard errors 

#Linear model with OLS
model1 = sm.OLS.from_formula(
    'BPXSY1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(RIDRETH1)',
    data = da
)
result1 = model1.fit()

#Linear model with GEE
model2 = sm.GEE.from_formula(
    'BPXSY1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(RIDRETH1)',
    groups = 'group',
    cov_struct = sm.cov_struct.Exchangeable(),
    data = da
)
result2 = model2.fit()

x = pd.DataFrame({
    'OLS_params': result1.params, 'OLS_SE': result1.bse,
    'GEE_params': result2.params, 'GEE_SE': result2.bse
})
x = x[['OLS_params', 'OLS_SE', 'GEE_params', 'GEE_SE']]

print(x)

                   OLS_params    OLS_SE  GEE_params    GEE_SE
Intercept           91.736583  1.339378   92.168530  1.384309
RIAGENDRx[T.Male]    3.671294  0.453763    3.650245  0.454498
C(RIDRETH1)[T.2]     0.855488  0.819486    0.159296  0.767025
C(RIDRETH1)[T.3]    -1.796132  0.671954   -2.233280  0.760228
C(RIDRETH1)[T.4]     3.813314  0.732355    3.105654  0.881580
C(RIDRETH1)[T.5]    -0.455347  0.808948   -0.439831  0.813675
RIDAGEYR             0.478699  0.012901    0.474101  0.018493
BMXBMI               0.278015  0.033285    0.280205  0.038553


The point estimates are similar between the OLS and GEE fits of the model, but the standard errors tend to be larger in the GEE fit. For example, the standard errors for BMI and age are 20-40% larger in the GEE fit. Since we know that there is dependence in these data that is driven by clustering, the OLS approach is not theoretically justified (the OLS parameter estimates remain in meaningful, but the standard errors do not). GEE parameter estimates and standard errors are meaningful in the presence of dependence, as long as the dependence is exclusively between observations within the same cluster.

### Marginal logistic model with dependent data

In [32]:
#GLM vs GEE standard errors 

#Relabelling education levels
da['DMDEDUC2x'] = da.DMDEDUC2.replace({
    1: "lt9", 2: "x9_11", 3: "HS", 4: "SomeCollege",
    5: "College", 7: np.nan, 9: np.nan
})

#Logistic model with GLM
model1 = sm.GLM.from_formula(
    formula = 'smq ~ RIDAGEYR + RIAGENDRx + C(DMDEDUC2x)',
    family = sm.families.Binomial(),
    data = da
)
result1 = model1.fit()

#Logistic model with GEE
model2 = sm.GEE.from_formula(
    formula = 'smq ~ RIDAGEYR + RIAGENDRx + C(DMDEDUC2x)',
    family = sm.families.Binomial(),
    groups = 'group',
    cov_struct = sm.cov_struct.Exchangeable(),
    data = da
)

result2 = model2.fit()

x = pd.DataFrame({"OLS_params": result1.params, "OLS_SE": result1.bse,
                  "GEE_params": result2.params, "GEE_SE": result2.bse})
x = x[["OLS_params", "OLS_SE", "GEE_params", "GEE_SE"]]
print(x)

                             OLS_params    OLS_SE  GEE_params    GEE_SE
Intercept                     -2.305999  0.114308   -2.249820  0.140567
RIAGENDRx[T.Male]              0.909597  0.060167    0.908682  0.062342
C(DMDEDUC2x)[T.HS]             0.943364  0.089663    0.887965  0.095397
C(DMDEDUC2x)[T.SomeCollege]    0.832227  0.084361    0.771636  0.104449
C(DMDEDUC2x)[T.lt9]            0.266228  0.109183    0.321784  0.141327
C(DMDEDUC2x)[T.x9_11]          1.098561  0.106697    1.062149  0.138401
RIDAGEYR                       0.018257  0.001725    0.017416  0.001803


The results show that the GLM and the GEE give very similar estimates for the regression parameters. However the standard errors obtained using GEE are somewhat larger than those obtained using GLM. This indicates that GLM understates the uncertainty in the estimated mean structure, which is a direct consequence of it ignoring the dependence structure. 

To the extent that the GLM and GEE parameter estimates differ, this is due to GEE attempting to exploit the dependence structure to obtain more efficient (i.e. more accurate) estimates of the model parameters. Thus, in summary, we can view GEE as trying to accomplish three things above and beyond what we obtain from GLM:

* GEE gives us insight into the dependence structure of the data

* GEE uses the dependence structure to obtain meaningful standard errors of the estimated model parameters.

* GEE uses the dependence structure to estimate the model parameters more accurately

### Multilevel linear models

A multilevel model is usually expressed in terms of random effects. These are variables that we do not observe, but that we can nevertheless incorporate into a statistical model. While these random effects are not observed, their presence can be inferred through the data, as long as each random effect is modeled as influencing at least two observations.

In [34]:
#Mixed effect linear model
model = sm.MixedLM.from_formula(
    "BPXSY1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(RIDRETH1)",
    groups = 'group',
    data = da
)

result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
           Mixed Linear Model Regression Results
============================================================
Model:             MixedLM  Dependent Variable:  BPXSY1     
No. Observations:  5102     Method:              REML       
No. Groups:        30       Scale:               256.6952   
Min. group size:   106      Log-Likelihood:      -21409.8702
Max. group size:   226      Converged:           Yes        
Mean group size:   170.1                                    
------------------------------------------------------------
                  Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------
Intercept         92.173    1.402 65.752 0.000 89.426 94.921
RIAGENDRx[T.Male]  3.650    0.452  8.084 0.000  2.765  4.535
C(RIDRETH1)[T.2]   0.153    0.887  0.172 0.863 -1.586  1.891
C(RIDRETH1)[T.3]  -2.238    0.758 -2.954 0.003 -3.723 -0.753
C(RIDRETH1)[T.4]   3.098    0.836  3.707 0.000  1.460  4.737
C(RIDRETH1)[T.5]  -0.439    0.878 -0.500 0.617 -2.161  1.282
RIDAGEYR           0.474    0.013 36.482 0.000  0.449  0.500
BMXBMI             0.280    0.033  8.404 0.000  0.215  0.346
group Var          3.615    0.085                           
============================================================

"""

The "variance structure parameters" are what distinguish a mixed model from a marginal model. Here we only have one such parameter, which is the variance for groups, estimated to be 3.615. This means that if we were to choose two groups at random, their random effects would differ on average by around 2.69 (this is calculated as the square root of 2*3.615). This is a sizable shift, comparable to the difference between females and males, or to around 6 years of aging.

The predicted random effects for the 30 groups in this analysis are shown below:

In [35]:
result.random_effects

{1191: group   -1.630976
 dtype: float64,
 1192: group   -0.086162
 dtype: float64,
 1201: group   -2.042661
 dtype: float64,
 1202: group   -0.147472
 dtype: float64,
 1211: group    0.280623
 dtype: float64,
 1212: group    1.580732
 dtype: float64,
 1221: group    0.283347
 dtype: float64,
 1222: group    0.131512
 dtype: float64,
 1231: group   -2.038171
 dtype: float64,
 1232: group    0.617651
 dtype: float64,
 1241: group    2.878488
 dtype: float64,
 1242: group   -0.519364
 dtype: float64,
 1251: group    2.064967
 dtype: float64,
 1252: group    1.521281
 dtype: float64,
 1261: group   -1.261975
 dtype: float64,
 1262: group    0.980846
 dtype: float64,
 1271: group    0.118031
 dtype: float64,
 1272: group   -0.128397
 dtype: float64,
 1281: group   -0.384862
 dtype: float64,
 1282: group   -3.582111
 dtype: float64,
 1291: group   -3.271017
 dtype: float64,
 1292: group   -0.829538
 dtype: float64,
 1301: group   -0.884171
 dtype: float64,
 1302: group    2.790657
 dtype: f

Based on these predicted random effects (also known as BLUPs), we see, for example, that cluster 1241 has unusually high SBP, and cluster 1282 has unusually low SBP. These deviations from what is expected are observed after adjusting for the covariates in the model, and hence are presumably driven by other characteristics of these clusters that are not reflected in the covariates.

### Random slopes

Multilevel modeling is a broad framework that allows many different types of models to be specified and fit. Here we are primarily focusing on the "random intercept models", that allow the data in each cluster to be shifted by a common amount, in order to account for stable confounders within clusters

Next we consider a more subtle form of cluster effect, in which the slope for a specific covariate varies from cluster to cluster. This is called a random slopes model. To demonstrate, we fit a model in which SBP has cluster-specific intercepts, and cluster-specific slopes for the age covariate. That is, we ask whether the rate at which blood pressure increases with age might differ from one cluster to the next.

In [37]:
#Centering age covariate (within clusters) for better convergence and robustness
da['age_cen'] = da.groupby('group').RIDAGEYR.transform(lambda x: x - x.mean())

#Fit model
model = sm.MixedLM.from_formula(
    "BPXSY1 ~ age_cen + RIAGENDRx + BMXBMI + C(RIDRETH1)",
    groups = 'group',
    vc_formula = {"age_cen": "0 + age_cen"},
    data = da
)
result = model.fit()
result.summary()

C:\Users\kevin\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
             Mixed Linear Model Regression Results
===============================================================
Model:              MixedLM   Dependent Variable:   BPXSY1     
No. Observations:   5102      Method:               REML       
No. Groups:         30        Scale:                263.7323   
Min. group size:    106       Log-Likelihood:       -21469.9240
Max. group size:    226       Converged:            Yes        
Mean group size:    170.1                                      
---------------------------------------------------------------
                   Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
---------------------------------------------------------------
Intercept         115.207    1.209 95.265 0.000 112.836 117.577
RIAGENDRx[T.Male]   3.643    0.457  7.962 0.000   2.746   4.539
C(RIDRETH1)[T.2]    1.167    0.827  1.412 0.158  -0.453   2.787
C(RIDRETH1)[T.3]   -1.659    0.679 -2.444 0.015  -2.989  -0.328
C(RIDRETH1)[T.4]    3.610    0.739  4.884 0.000   2.161   5.058
C(RIDRETH1)[T.5]   -1.208    0.816 -1.480 0.139  -2.807   0.392
age_cen             0.467    0.018 26.235 0.000   0.432   0.502
BMXBMI              0.288    0.034  8.574 0.000   0.222   0.353
age_cen Var         0.004    0.000                             
===============================================================

"""

In [40]:
std_dev = np.round(np.sqrt(0.004), 2)
std_dev

0.06

We see that the estimated variance for random age slopes is 0.004, which translates to a standard deviation of 0.06. That is, from one cluster to another, the age slopes fluctuate by  ±0.06−0.12  (1-2 standard deviations). These cluster-specific fluctuations are added/subtracted from the fixed effect for age, which is 0.467. Thus, in some clusters SBP may increase by around 0.467 + 0.06 = 0.527 mm/Hg per year, while in other clusters SBP may increase by only around 0.467 - 0.06 = 0.407 mm/Hg per year.

Next, we fit a model in which the cluster-specific intercepts and slopes are allowed to be correlated.

In [43]:
model = sm.MixedLM.from_formula(
    "BPXSY1 ~ age_cen + RIAGENDRx + BMXBMI + C(RIDRETH1)",
    groups = "group", 
    re_formula = "1 + age_cen", 
    data = da
)
result = model.fit()
result.summary()

C:\Users\kevin\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
              Mixed Linear Model Regression Results
=================================================================
Model:                MixedLM   Dependent Variable:   BPXSY1     
No. Observations:     5102      Method:               REML       
No. Groups:           30        Scale:                255.4451   
Min. group size:      106       Log-Likelihood:       -21413.6193
Max. group size:      226       Converged:            Yes        
Mean group size:      170.1                                      
-----------------------------------------------------------------
                     Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
-----------------------------------------------------------------
Intercept           115.467    1.340 86.173 0.000 112.840 118.093
RIAGENDRx[T.Male]     3.662    0.451  8.121 0.000   2.778   4.546
C(RIDRETH1)[T.2]      0.023    0.898  0.025 0.980  -1.738   1.783
C(RIDRETH1)[T.3]     -2.251    0.778 -2.893 0.004  -3.775  -0.726
C(RIDRETH1)[T.4]      3.011    0.854  3.524 0.000   1.336   4.686
C(RIDRETH1)[T.5]     -0.585    0.893 -0.655 0.512  -2.336   1.165
age_cen               0.466    0.018 26.286 0.000   0.431   0.501
BMXBMI                0.283    0.033  8.497 0.000   0.218   0.349
group Var             8.655    0.169                             
group x age_cen Cov   0.119    0.004                             
age_cen Var           0.004    0.000                             
=================================================================

"""

The estimated correlation coefficient between random slopes and random intercepts is estimated to be 0.119/sqrt(8.655*0.004), which is around 0.64. This indicates that the clusters with unusually high average SBP also tend to have SBP increasing faster with age. Note however that these structural parameters are only estimates, and due to the variance parameter falling close to the boundary, the estimates may not be particularly precise.